# Step 1: Create model with transfer learning

In [4]:
# Function to create I3D model
from i3d_inception import Inception_Inflated3d

In [5]:
# Parameters for the model
NUM_FRAMES = 79
FRAME_HEIGHT = 224
FRAME_WIDTH = 224
NUM_FLOW_CHANNELS = 2
NUM_CLASSES = 2

In [6]:
# Create the I3D model without top layer and using pretrained weights
flow_model = Inception_Inflated3d(
                include_top=False,
                weights='flow_imagenet_and_kinetics',
                input_shape=(None, FRAME_HEIGHT, FRAME_WIDTH, NUM_FLOW_CHANNELS),
                classes=NUM_CLASSES)

49512448/49507528 [==============================] - 2s 0us/step


In [7]:
# Summary of the model
flow_model.summary()

Model: "i3d_inception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 224, 2 0                                            
__________________________________________________________________________________________________
Conv3d_1a_7x7_conv (Conv3D)     (None, None, 112, 11 43904       input_2[0][0]                    
__________________________________________________________________________________________________
Conv3d_1a_7x7_bn (BatchNormaliz (None, None, 112, 11 192         Conv3d_1a_7x7_conv[0][0]         
__________________________________________________________________________________________________
Conv3d_1a_7x7 (Activation)      (None, None, 112, 11 0           Conv3d_1a_7x7_bn[0][0]           
______________________________________________________________________________________

# Step 2: Add top layers (classification layers)

In [8]:
from keras import models
from keras import layers
from keras import backend as K

In [9]:
# Parameters
dropout_prob = 0.0

In [11]:
# Creating sequential model
model = models.Sequential()
model.add(flow_model)

# Adding top layers
model.add(layers.Dropout(dropout_prob))
model.add(layers.Conv3D(NUM_CLASSES, (1, 1, 1), 
                        strides = (1, 1, 1), 
                        padding = 'same',
                        use_bias = False,
                        name = 'Conv3d_6a_1x1'))

model.add(layers.Reshape((-1, NUM_CLASSES)))

model.add(layers.Lambda(lambda x: K.mean(x, axis=1, keepdims=False),
                        output_shape=lambda s: (s[0], s[2])))

model.add(layers.Activation('softmax', name = 'prediction'))

In [12]:
# Make only the last two inception modules and top layer trainable
for layer in flow_model.layers[:-21]:
    layer.trainable = False

In [13]:
# Summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
i3d_inception (Functional)   (None, None, 1, 1, 1024)  12272592  
_________________________________________________________________
dropout (Dropout)            (None, None, 1, 1, 1024)  0         
_________________________________________________________________
Conv3d_6a_1x1 (Conv3D)       (None, None, 1, 1, 2)     2048      
_________________________________________________________________
reshape (Reshape)            (None, None, 2)           0         
_________________________________________________________________
lambda (Lambda)              (None, 2)                 0         
_________________________________________________________________
prediction (Activation)      (None, 2)                 0         
Total params: 12,274,640
Trainable params: 2,785,520
Non-trainable params: 9,489,120
___________________________________

# Step 3: Create data generators

In [14]:
# Import custom data generator class
from i3ddatagenerator import DataGenerator

In [15]:
# Parameters
batch_size = 8
path_train = '../datai3d/rgb/train/'
path_val = '../datai3d/rgb/validation/'
path_test = '../datai3d/rgb/test/'

In [16]:
# Create data generators for training, validation and test
train_generator = DataGenerator(directory=path_train,
                                batch_size=batch_size, 
                                data_augmentation=True)

validation_generator = DataGenerator(directory=path_val,
                                     batch_size=batch_size,
                                     data_augmentation=False,
                                     target_frames = None,
                                     flip = False)

test_generator = DataGenerator(directory=path_test,
                               shuffle = False,
                               batch_size=batch_size, 
                               data_augmentation=False,
                               target_frames = None,
                               flip = False)

Found 1810 files belonging to 2 classes.
     Fight :  0
  NonFight :  1
Found 393 files belonging to 2 classes.
     Fight :  0
  NonFight :  1
Found 400 files belonging to 2 classes.
     Fight :  0
  NonFight :  1


# Step 4: Train RGB model

In [ ]:
from keras import optimizers

In [16]:
# Parameters
epochs = 100
steps_per_epoch = train_generator.n_files//batch_size
validation_steps = validation_generator.n_files//batch_size

# Path to store checkpoints
filepath = 'checkpoints/weights_i3dflow.hdf5'

In [17]:
# Callbacks

# Reduce learning rate by 10x when plateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                 factor=0.1,
                                                 patience=5, 
                                                 min_lr=0.0001,
                                                verbose = 1)

# Save best model
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, 
                                                monitor='val_loss', 
                                                verbose=1, 
                                                save_best_only=True, 
                                                save_weights_only=False, 
                                                mode='auto', 
                                                save_freq='epoch')

# Stop after 10 epochs if val loss does not improve
earlystop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', 
                                             restore_best_weights = True, 
                                             patience = 10, 
                                             min_delta = 0.01,
                                            verbose = 1)

callbacks = [reduce_lr, checkpoint, earlystop]

In [18]:
# Create optimiser (SGD with momentum)
optimizer = keras.optimizers.SGD(learning_rate = 0.01, momentum = 0.9)

# Compile model
model.compile(optimizer=optimizer, loss='CategoricalCrossentropy', metrics=['accuracy'])

In [19]:
# Start training and save history
history = model.fit(
      train_generator,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_data=validation_generator,
      validation_steps=validation_steps,
      callbacks = callbacks,
      verbose=1)

Epoch 1/100
150/150 [==============================] - 1251s 8s/step - loss: 0.6255 - accuracy: 0.6717 - val_loss: 0.5947 - val_accuracy: 0.6964

Epoch 00001: val_loss improved from inf to 0.59468, saving model to checkpoints\weights_i3drgb.hdf5
Epoch 2/100
150/150 [==============================] - 1258s 8s/step - loss: 0.5258 - accuracy: 0.7540 - val_loss: 0.5573 - val_accuracy: 0.7423

Epoch 00002: val_loss improved from 0.59468 to 0.55733, saving model to checkpoints\weights_i3drgb.hdf5
Epoch 3/100
150/150 [==============================] - 1273s 8s/step - loss: 0.5225 - accuracy: 0.7352 - val_loss: 0.5675 - val_accuracy: 0.6913

Epoch 00003: val_loss did not improve from 0.55733
Epoch 4/100
150/150 [==============================] - 1226s 8s/step - loss: 0.4659 - accuracy: 0.7867 - val_loss: 0.6052 - val_accuracy: 0.7041

Epoch 00004: val_loss did not improve from 0.55733
Epoch 5/100
150/150 [==============================] - 1236s 8s/step - loss: 0.4928 - accuracy: 0.7795 - val_l

In [20]:
# Save history of the model
import pickle
with open('history_i3dflow', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

# Step 5: Test model

In [17]:
# Parameters
test_steps = test_generator.n_files//batch_size

In [23]:
# Evaluate model on test set
model.evaluate(test_generator,
               steps = test_steps,
               return_dict =  True)

50/50 [==============================] - 346s 7s/step - loss: 0.4474 - accuracy: 0.7775


{'loss': 0.4473906457424164, 'accuracy': 0.7774999737739563}